In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
import os.path
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
from fastai.imports import *
# Veri seti klasörünün yolu (Data klasörünüzün yolu)
data_dir = Path('/content/drive/MyDrive/Datas')

filepaths = list(data_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)

train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2) # %70 eğitim datası içinden ön eğitim kontrölü içinde %20 yani 140 adet ön test (daha önceden gördüğü veri üzerinden test yapmak için) veri ayırıyoruz

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(100,100),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(100,100),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(100,100),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)


Found 560 validated image filenames belonging to 4 classes.
Found 140 validated image filenames belonging to 4 classes.
Found 300 validated image filenames belonging to 4 classes.


In [ ]:
pretrained_model = tf.keras.applications.resnet50.ResNet50(
                    input_shape=(100,100,3), # 100X100 resimde 3 girişli bir kanal (ağ) yapısı vardır standart ölçü
                    include_top=False,# katmanın başlangıç noktası (en üst kısmıdır) veri kalitesi ve eğitim açısından eklenmez bu yüzden False ise eklenmeyecek ise tanımlamak zorundayız
                    weights='imagenet',
                    pooling='avg')

pretrained_model.trainable = False

inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(50, activation='relu')(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x) # 4 sınıf sayısıdır
model = tf.keras.Model(inputs, outputs)

94765736/94765736 [==============================] - 2s 0us/step


In [ ]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 106, 106, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 50, 50, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 50, 50, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3,restore_best_weights=True,)
history = model.fit(train_images, validation_data=val_images, epochs=10, callbacks=[callbacks])


Epoch 1/10
18/18 [==============================] - 47s 2s/step - loss: 0.0493 - accuracy: 0.9875 - val_loss: 0.5280 - val_accuracy: 0.8643
Epoch 2/10
18/18 [==============================] - 35s 2s/step - loss: 0.0233 - accuracy: 0.9964 - val_loss: 0.6450 - val_accuracy: 0.8643
Epoch 3/10
18/18 [==============================] - 37s 2s/step - loss: 0.0176 - accuracy: 0.9946 - val_loss: 0.6097 - val_accuracy: 0.8714
Epoch 4/10
18/18 [==============================] - 36s 2s/step - loss: 0.0095 - accuracy: 0.9982 - val_loss: 0.6097 - val_accuracy: 0.8786
Epoch 5/10
18/18 [==============================] - 32s 2s/step - loss: 0.0148 - accuracy: 0.9946 - val_loss: 0.6729 - val_accuracy: 0.8500
Epoch 6/10
18/18 [==============================] - 37s 2s/step - loss: 0.0046 - accuracy: 0.9982 - val_loss: 0.5989 - val_accuracy: 0.8643
Epoch 7/10
18/18 [==============================] - 39s 2s/step - loss: 0.0108 - accuracy: 0.9982 - val_loss: 0.6181 - val_accuracy: 0.8857
Epoch 8/10
18/18 [==

In [ ]:
results = model.evaluate(test_images, verbose=0)
print(results)
print(f"Test Accuracy: {np.round(results[1] * 100,2)}%")

In [ ]:
predictions = np.argmax(model.predict(test_images), axis=1)
matrix = confusion_matrix(test_images.labels, predictions)
report= classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

In [ ]:
fig = plt.figure(figsize=(8, 8))
sns.heatmap(matrix, annot=True, cmap='viridis')
plt.xticks(ticks=np.arange(4) + 0.5, labels=test_images.class_indices.keys(), rotation=90)
plt.yticks(ticks=np.arange(4) + 0.5, labels=test_images.class_indices.keys(), rotation=0)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
fig.savefig("Confusion Matrix",dpi=200)

In [ ]:
print("Classification Report:\n", report)